<a href="https://colab.research.google.com/github/snpushpi/MIT_machine_learning/blob/master/Copy_of_MIT_6_036_HW09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MIT 6.036 Spring 2020: Homework 9#

This colab notebook provides code and a framework for question 1 and 5 of the [homework](https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2020_Spring/courseware/Week9/week9_homework/).  You can work out your solutions here, then submit your results back on the homework page when ready.


## Setup

First, download the code distribution for this homework that contains test cases and helper functions.

Run the next code block to download and import the code for this lab.

In [0]:
!rm -rf code_for_hw9*
!wget --quiet https://introml.odl.mit.edu/cat-soop/_static/6.036/homework/hw09/code_for_hw9.zip
!unzip code_for_hw9.zip
!mv code_for_hw9/* .

from dist import *
from sm import *
from util import *
from mdp import *

import mdp
import numpy as np

Archive:  code_for_hw9.zip
   creating: code_for_hw9/
  inflating: code_for_hw9/util.py    
   creating: __MACOSX/
   creating: __MACOSX/code_for_hw9/
  inflating: __MACOSX/code_for_hw9/._util.py  
  inflating: code_for_hw9/sm.py      
  inflating: __MACOSX/code_for_hw9/._sm.py  
  inflating: code_for_hw9/mdp.py     
  inflating: __MACOSX/code_for_hw9/._mdp.py  
  inflating: code_for_hw9/tests.py   
  inflating: __MACOSX/code_for_hw9/._tests.py  
  inflating: code_for_hw9/dist.py    
  inflating: __MACOSX/code_for_hw9/._dist.py  
  inflating: __MACOSX/._code_for_hw9  
   creating: code_for_hw9/__pycache__/
  inflating: code_for_hw9/__pycache__/mdp.cpython-38.pyc  
  inflating: code_for_hw9/__pycache__/sm.cpython-38.pyc  
  inflating: code_for_hw9/__pycache__/dist.cpython-38.pyc  
  inflating: code_for_hw9/__pycache__/util.cpython-38.pyc  


## 1) State Machines

We will implement state machines as sub-classes of the `SM` class, which specifies the `start_state`, `transition_fn` and `output_fn`.

```
class SM:
    start_state = None  # default start state
    def transition_fn(self, s, x):
        '''s:       the current state
           i:       the given input
           returns: the next state'''
        raise NotImplementedError
    def output_fn(self, s):
        '''s:       the current state
           returns: the corresponding output'''
        raise NotImplementedError
```

An example of a sub-class is the `Accumulator` state machine, which adds up (accumulates) its input and outputs the sum. Convince yourself that the implementation works as expected before moving on.

```
class Accumulator(SM):
    start_state = 0
    def transition_fn(self, s, x):
        return s + x
    def output_fn(self, s):
        return s
```

### 1.1 Transduce
Implement the `transduce` method for the `SM` class. It is given an input sequence (a list) and returns an output sequence (a list) of the outputs of the state machine on the input sequence. Assume `self.transition_fn` and `self.output_fn` are defined.

In [0]:
class SM:
    start_state = None
    def transduce(self, input_seq):
        '''input_seq: a list of inputs to feed into SM
           returns:   a list of outputs of SM'''
        output_seq = []
        curr_state = self.start_state
        for i in input_seq:
            curr_state = self.transition_fn(curr_state,i)
            output_seq.append(self.output_fn(curr_state))
        return output_seq

Below is the `Accumulator` state machine implementation that you saw above as well as an unit test to help test your `SM` class.

In [0]:
class Accumulator(SM):
    start_state = 0

    def transition_fn(self, s, x):
        return s + x

    def output_fn(self, s):
        return s
    
def test_accumulator_sm():
    res = Accumulator().transduce([-1, 2, 3, -2, 5, 6])
    assert(res == [-1, 1, 4, 2, 7, 13])
    print("Test passed!")

# Unit test
test_accumulator_sm()

Test passed!


### 1.2 Binary Addition
Implement a `Binary_Addition` state machine that takes in a sequence of pairs of binary digits (0,1) representing two reversed binary numbers and returns a sequence of digits representing the reversed sum. For instance, to sum two binary numbers `100` and `011`, the input sequence will be `[(0, 1), (0, 1), (1, 0)]`. You will need to define `start_state`, `transition_fn` and `output_fn`. Note that when transduced, the input sequence may need to be extended with an extra (0,0) to output the final carry.

In [0]:
class Binary_Addition(SM):
    

    start_state = [0,0]#left side is carry, right side is sum of two bits
    
    def transition_fn(self, s, x):
        # Your code here
        #the second part of the tuple is remainder
        if s[0]==0:
            if (x[0]==0 and x[1]==1) or (x[0]==1 and x[1]==0):
                s[1]=1
                
            elif x[1]==1 and x[1]==1:
                s[0]=1
                s[1]=0
            else:
                s[1]=0
                s[0]=0
        else:
            if (x[0]==0 and x[1]==1) or (x[0]==1 and x[1]==0):
                s[1]=0
            elif x[0]==1 and x[1]==1:
                s[0]=1
                s[1]=1
            else:
                s[0]=0
                s[1]=1
                
        return s         
            
            
    def output_fn(self, s):
        # Your code here
        print('hi',s)
        return s[1]

In [0]:
def test_binary_addition_sm():
    res = Binary_Addition().transduce([(1, 1), (1, 0), (0, 0)])
    assert(res == [0, 0, 1])
    print("Test passed!")

# Unit test
test_binary_addition_sm()

hi [1, 0]
hi [1, 0]
hi [0, 1]
Test passed!


### 1.3 Reverser
Implement a state machine that reverses a sequence. The input is a list of the form:

```
 sequence1 + ['end'] + sequence2
 ```
 
`+` refers to concatenation. `sequence1` is a list of strings, the `'end'` string indicates termination, and `sequence2` is arbitrary. The machine reverses `sequence1`: for each entry in the `sequence1`, the machine outputs `None`. For the `'end'` input and each entry in the second sequence, an item from the reversed `sequence1` is output, or `None` if no characters remain.

In [0]:
class Reverser(SM):
    start_state = [1]

    def transition_fn(self, s, x):
        # Your code here
        if x=='end' and s[0]==2:
            if len(s)>1:
                s.pop()
                return s
        if x=='end':
            s[0]=2 
            return s
        if s[0]==1:
            s.append(x) 
            return s
        if x!='end' and s[0]==2:
            if len(s)>1:
                s.pop() 
        
        return s 

    def output_fn(self, s):
        # Your code here
        print(s)
        if len(s)>1:
            if s[0]==1:
                return None
            else:
                return s[-1]
        else:
            return None
        

In [0]:
def test_reverser_sm():
    res = Reverser().transduce(['foo', ' ', 'bar'] + ['end'] + list(range(5)))
    assert(res == [None, None, None, 'bar', ' ', 'foo', None, None, None])
    print("Test passed!")

# Unit test
test_reverser_sm()

[1, 'foo']
[1, 'foo', ' ']
[1, 'foo', ' ', 'bar']
[2, 'foo', ' ', 'bar']
[2, 'foo', ' ']
[2, 'foo']
[2]
[2]
[2]
Test passed!


### 1.4 RNN
An RNN has a transition function and an output function, each of which is defined in terms of weight matrices, offset vectors and activation functions, analogously to standard neural networks.

* The inputs $x$ are $l\times1$ vectors
* The states $s$ are $m\times1$ vectors
* The outputs $y$ are $n\times1$ vectors

The behavior is defined as follows:
$$\begin{align*} s_{t} & = f_1(W^{ss} s_{{t-1}} + W^{sx} x_{t} + W^{ss}_0) \\ y_{t} & = f_2(W^o s_{t} + W^o_0) \end{align*}$$

where $f_1$ and $f_2$ are two activation functions, such as linear, softmax or tanh.


Note that each input `i` below has dimension `l x 1`. Implement the corresponding state machine, where the weights are given in `__init__`. Make sure to set an appropriate `start_state`.

In [0]:
class RNN(SM):
    def __init__(self, Wsx, Wss, Wo, Wss_0, Wo_0, f1, f2, start_state):
        # Your code here
        self.start_state = start_state
        self.Wsx = Wsx
        self.Wss = Wss
        self.Wo = Wo
        self.Wss_0 = Wss_0
        self.Wo_0 = Wo_0
        self.f1 = f1
        self.f2 = f2


    def transition_fn(self, s, x):
        mat_1 = np.matmul(self.Wss,s)
        mat_2 = np.matmul(self.Wsx,x)
        return self.f1(mat_1+mat_2+self.Wss_0)
        

    def output_fn(self, s):
        mat1= np.matmul(self.Wo,s)
        return self.f2(mat1+self.Wo_0)

In [0]:
def softmax(z):
    v = np.exp(z)
    return v / np.sum(v, axis = 0
Wsx1 = np.array([[0.1],
                     [0.1],
                     [0.1]])
Wss1 = np.array([[0.1,0.2,0.3],
                     [0.4,0.5,0.6],
                     [0.7,0.8,0.9]])
Wo1 = np.array([[0.1,0.2,0.3]]
                    )
Wss1_0 = np.array([[0.01],
                       [0.02],
                       [0.03]])
Wo1_0 = np.array([[0.1]])
    
start_state = np.array([[0,0,0]]).T
rnn = RNN(Wsx1, Wss1, Wo1, Wss1_0, Wo1_0, np.tanh, softmax, start_state)
rnn.transduce([np.array([[-1]]), np.array([[-2]]), np.array([[2]]),np.array([[3]]), np.array([[-3]]), np.array([[1]])]) 

NameError: ignored

In [0]:

def test_rnn():
    Wsx1 = np.array([[0.1],
                     [0.1],
                     [0.1]])
    Wss1 = np.array([[0.1,0.2,0.3],
                     [0.4,0.5,0.6],
                     [0.7,0.8,0.9]])
    Wo1 = np.array([[0.1,0.2,0.3],
                    [0.4,0.5,0.6]])
    Wss1_0 = np.array([[0.01],
                       [0.02],
                       [0.03]])
    Wo1_0 = np.array([[0.1],
                      [0.2]])
    in1 = [np.array([[0.1]]),
           np.array([[0.3]]),
           np.array([[0.5]])]
    start_state = np.array([[0,0,0]]).T
    
    rnn = RNN(Wsx1, Wss1, Wo1, Wss1_0, Wo1_0, np.tanh, softmax, start_state)
    expected = np.array([[[0.4638293846951024], [0.5361706153048975]],
                        [[0.4333239107898491], [0.566676089210151]],
                        [[0.3821688606165438], [0.6178311393834561]]])

    assert(np.allclose(expected, rnn.transduce(in1)))
    print("Test passed!")

# Unit test
test_rnn()

### 1.5 Accumulator Sign RNN
Enter the parameter matrices and vectors for an instance of the `RNN` class such that the output is `1` if the cumulative sum of the inputs is positive, `-1` if the cumulative sum is negative and `0` if otherwise. Make sure that you scale the outputs so that the output activation values are very close to `1`, `0` and `-1`. Note that both the inputs and outputs are `1 x 1`.

Hint: `np.tanh` may be useful. Remember to convert your Python lists to `np.array`.

In [0]:
def tanh_checker(x):
    if np.tanh(x)<0:
        if np.tanh(x)>-0.01:
            return 0
        else:
            return -1
    elif np.tanh(x)>0:
        return 1
    else:
        return 0
f = np.vectorize(tanh_checker)
Wsx = np.array([[1/3],
                     [1/3],
                     [1/3]])
Wss = np.array([[1/3,1/3,1/3],
                     [1/3,1/3,1/3],
                     [1/3,1/3,1/3]])
Wo = np.array([[1/3,1/3,1/3]]
                    )
Wss_0 = np.array([[0],
                       [0],
                       [0]])
Wo_0 = np.array([[0]])
f1 = lambda x:x
f2 = f
start_state = np.array([[0,0,0]]).T
acc_sign = RNN(Wsx, Wss, Wo, Wss_0, Wo_0, f1, f2, start_state)

In [0]:
def test_acc_sign_rnn(acc_sign_rnn):
    res = acc_sign_rnn.transduce([-1, -2, 2, 3, -3, 1])
    print(res)
    expected = np.array([[[-1.0]], [[-1.0]], [[-1.0]], [[1.0]], [[-1.0]], [[0.0]]])
    assert(np.allclose(expected, res))
    print("Test passed!")

# Unit test
test_acc_sign_rnn(acc_sign)

### 1.6 Autoregression RNN

Enter the parameter matrices and vectors for an instance of the `RNN` class such that it implements the following autoregressive model:
$$y_t=y_{t-1} - 2y_{t-2} + 3y_{t-3}$$
Set `start_state` such that $y_1=-2$, $y_t=0$ for $t\lt1$. Note that all inputs will be zero.


In [0]:
Wsx =             np.zeros((3,1))
Wss =             np.array([[1,-2,3],[1,0,0],[0,1,0]])
Wo =              np.array([[1,-2,3]])
Wss_0 =           np.zeros((3,1))
Wo_0 =            np.zeros((1,1))
f1 =              lambda x:x
f2 =              lambda x:x
start_state =     np.array([[0,0,-2/3]]).T
auto = RNN(Wsx, Wss, Wo, Wss_0, Wo_0, f1, f2, start_state)


In [0]:
def test_auto_rnn(auto_rnn):
    res = auto_rnn.transduce([np.array([[0]]) for x in range(5)])
    expected = np.array([[[-2.0]], [[2.0]], [[0.0]], [[-10.0]], [[-4.0]]])
    assert(np.allclose(expected, res))
    print("Test passed!")
    
# Unit test
test_auto_rnn(auto)

## 5) MDP Implementations

We'll be using a couple of simple classes to represent MDPs and probability distributions.

###5.1 Working with MDPs

Recall that given a $Q_\pi$ for any policy $\pi$, then $V_\pi(s)$ = $\max_a Q_\pi(s, a)$.

1. Write the `value` method, which takes a $Q$ function (an instance of `TabularQ`) and a state and returns the value `V` of an action that maximizes $Q$ function stored in `q`.



In [0]:
def value(q, s):
    """ Return Q*(s,a) based on current Q

    >>> q = TabularQ([0,1,2,3],['b','c'])
    >>> q.set(0, 'b', 5)
    >>> q.set(0, 'c', 10)
    >>> q_star = value(q,0)
    >>> q_star
    10
    """
    max = None
    for action in q.actions:
        if max== None:
            max = q.q[(s,action)] 
        elif max< q.q[(s,action)]:
            max = q.q[(s,action)]
    return max
    


In [0]:
def test_value():
    q = TabularQ([0,1,2,3], ['b','c'])
    q.set(0, 'b', 5)
    q.set(0, 'c', 10)
    assert(value(q, 0) == 10)
    print("Test passed!")
    
test_value()

2. Write the `greedy` method, which takes a $Q$ function (an instance of `TabularQ`) and a state and returns the action `a` determined by the policy that acts greedily with respect to the current value of `q`.

In [0]:
def greedy(q, s):
    """ Return pi*(s) based on a greedy strategy.

    >>> q = TabularQ([0,1,2,3],['b','c'])
    >>> q.set(0, 'b', 5)
    >>> q.set(0, 'c', 10)
    >>> q.set(1, 'b', 2)
    >>> greedy(q, 0)
    'c'
    >>> greedy(q, 1)
    'b'
    """
    # Your code here
    m_action = None
    max = None
    for action in q.actions:
        if (s,action) in q.q:
            if m_action==None:
                max = q.q[(s,action)]
                m_action = action
            else:
                if q.q[(s,action)]>max:
                    max = q.q[(s,action)]
                    m_action = action
    return m_action


In [0]:
def test_greedy():
    q = TabularQ([0, 1, 2, 3],['b', 'c'])
    q.set(0, 'b', 5)
    q.set(0, 'c', 10)
    q.set(1, 'b', 2)
    assert(greedy(q, 0) == 'c')
    assert(greedy(q, 1) == 'b')
    print("Test passed!")

test_greedy()

3. Write the `epsilon_greedy` method, which takes a state `s` and a parameter `epsilon`, and returns an action. With probability `1 - epsilon` it should select the greedy action and with probability `epsilon` it should select an action uniformly from the set of possible actions.

    - You should use `random.random()` to generate a random number to test againts eps.
    - You should use the `draw` method of `uniform_dist` to generate a random action.
    - You can use the `greedy` function defined earlier.

In [0]:
def epsilon_greedy(q, s, eps = 0.5):
    """ Returns an action.

    >>> q = TabularQ([0,1,2,3],['b','c'])
    >>> q.set(0, 'b', 5)
    >>> q.set(0, 'c', 10)
    >>> q.set(1, 'b', 2)
    >>> eps = 0.
    >>> epsilon_greedy(q, 0, eps) #greedy
    'c'
    >>> epsilon_greedy(q, 1, eps) #greedy
    'b'
    """
    if random.random() < eps:  # True with prob eps, random action
        DDist_obj = uniform_dist(q.actions)
        return DDist_obj.draw()
    else:
        return greedy(q,s)
        

In [0]:
def test_epsilon_greedy():
    q = TabularQ([0, 1, 2, 3],['b', 'c'])
    q.set(0, 'b', 5)
    q.set(0, 'c', 10)
    q.set(1, 'b', 2)
    eps = 0.0
    assert(epsilon_greedy(q, 0, eps) == 'c')
    assert(epsilon_greedy(q, 1, eps) == 'b')
    print("Test passed!")
    
test_epsilon_greedy()

### 5.2 Implement Q-Value Iteration
Provide the definition of the `value_iteration` function. It takes an MDP instance and a `TabularQ` instance. It should terminate when

$$\max_{(s, a)}\left|Q_t(s, a) - Q_{t-1}(s, a)\right| < \epsilon$$

that is, the biggest difference between the value functions on successive iterations is less than input parameter `eps`. This function should return the final `TabularQ` instance. It should do no more that `max_iters` iterations.

* Make sure to copy the Q function between iterations, e.g. `new_q = q.copy()`.
* The `q` parameter contains the initialization of the Q function.
* The `value` function is already defined.

In [0]:
def value_iteration(mdp, q, eps = 0.01, max_iters=1000):
    for i in range(max_iters):
        q_new = q.copy()
        max = 0
        for elt in q.q:
            v = mdp.reward_fn(elt[0],elt[1])+mdp.discount_factor*mdp.transition_model(elt[0],elt[1]).expectation(lambda s:value(q,s))
            q_new.set(elt[0],elt[1],v)
            if max < abs(q_new.q[(elt[0],elt[1])]-q.q[(elt[0],elt[1])]):
                max = abs(q_new.q[(elt[0],elt[1])]-q.q[(elt[0],elt[1])])
        
        if max<eps:
            return q_new
        q = q_new.copy()
            

Below is the implementation of the "tiny" MDP detailed in Problem 2 and Problem 5.3. We will be using it to test `value_iteration`.

In [0]:
def tiny_reward(s, a):
    # Reward function
    if s == 1: return 1
    elif s == 3: return 2
    else: return 0

def tiny_transition(s, a):
    # Transition function
    if s == 0:
        if a == 'b':
            return DDist({1 : 0.9, 2 : 0.1})
        else:
            return DDist({1 : 0.1, 2 : 0.9})
    elif s == 1:
        return DDist({1 : 0.1, 0 : 0.9})
    elif s == 2:
        return DDist({2 : 0.1, 3 : 0.9})
    elif s == 3:
        return DDist({3 : 0.1, 0 : 0.9})
    
def test_value_iteration():
    tiny = MDP([0, 1, 2, 3], ['b', 'c'], tiny_transition, tiny_reward, 0.9)
    q = TabularQ(tiny.states, tiny.actions)
    qvi = value_iteration(tiny, q, eps=0.1, max_iters=100)
    expected = dict([((2, 'b'), 5.962924188028282),
                     ((1, 'c'), 5.6957634856549095),
                     ((1, 'b'), 5.6957634856549095),
                     ((0, 'b'), 5.072814297918393),
                     ((0, 'c'), 5.262109602844769),
                     ((3, 'b'), 6.794664584556008),
                     ((3, 'c'), 6.794664584556008),
                     ((2, 'c'), 5.962924188028282)])
    for k in qvi.q:
        print("k=%s, expected=%s, got=%s" % (k, expected[k], qvi.q[k]))      
        assert(abs(qvi.q[k] - expected[k]) < 1.0e-5)
    print("Test passed!")

test_value_iteration()

### 5.3 Receding-horizon control and online search
Write a procedure `q_em(mdp, s, a, h)` that computes the horizon-h Q value for state `s` and action `a` by using the definition of the finite-horizon Q function in the notes (but including a discount factor). 

This can be written as a relatively simple recursive procedure with a base case (what is the Q value when horizon is 0?) and a recursive case that computes the horizon `h` values assuming we can (recursively) get horizon `h-1` values.

In [0]:
def q_em(mdp, s, a, h):
    # Your code here
    if h==0:
        return 0
    return mdp.reward_fn(s,a)+mdp.discount_factor*mdp.transition_model(s,a).expectation(lambda s:q_em(mdp, s, a, h-1))

We will be using the "tiny" MDP again to test `q_em`.

In [0]:
def test_q_em():
    tiny = MDP([0, 1, 2, 3], ['b', 'c'], tiny_transition, tiny_reward, 0.9)
    assert(np.allclose([q_em(tiny, 0, 'b', 1)], [0.0]))
    assert(np.allclose([q_em(tiny, 0, 'b', 2)], [0.81]))
    assert(np.allclose([q_em(tiny, 0, 'b', 3)], [1.0287000000000002]))
    assert(np.allclose([q_em(tiny, 0, 'c', 3)], [1.4103]))
    assert(np.allclose([q_em(tiny, 2, 'b', 3)], [1.9116000000000002]))
    print("Tests passed!")

test_q_em()